In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Table, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json
import openai
import json
from pydantic import Field
from typing import List
import os
from openai_schema import OpenAISchema

# Create a declarative base class
Base = declarative_base()

# Define a dynamic query class
class DynamicQuery:
    def __init__(self):
        self._table = None
        self._columns = []

    @property
    def table(self):
        return self._table

    @table.setter
    def table(self, table_name):
        self._table = table_name

    @property
    def columns(self):
        return self._columns

    @columns.setter
    def columns(self, columns_list):
        self._columns = columns_list

    def execute_query(self, engine):
        if not self.table or not self.columns:
            raise ValueError("Table and columns must be set before executing the query.")

        metadata = MetaData(bind=engine)
        metadata.reflect()
        table = metadata.tables.get(self.table)

        if table is None:
            raise ValueError(f"Table '{self.table}' does not exist.")

        select_query = table.select().with_only_columns([table.c[col] for col in self.columns])
        with engine.connect() as connection:
            result = connection.execute(select_query)
            return result.fetchall()

# Example usage
engine = create_engine('sqlite:///your_database.db')
dynamic_query = DynamicQuery()
dynamic_query.table = 'your_table_name'  # Set the table name
dynamic_query.columns = ['column1', 'column2']  # Set the columns

# Execute the query
results = dynamic_query.execute_query(engine)
for row in results:
    print(row)
